<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/2_GPT_multiclass_batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP final - reconocimiento de discursos discriminatorios en Twitter

## Clasificacion multietiquetas con GPT 4 mini (sin finetunning) por batch


**Natalia Dedandi**





La documentación de la API se encuentra aqui:

https://platform.openai.com/docs/api-reference/introduction



In [76]:
pip install openai

In [77]:
!pip install datasets seaborn


In [78]:
import json
from openai import OpenAI
import os
from google.colab import userdata
#seteo la KEY
os.environ['OPENAI_API_KEY'] = userdata.get('openIA_key')




In [79]:
#creo el cliente OPENAI con mi usuario y proyecto
client = OpenAI(
  organization='org-1uHjwiaB3OlPzoxfVzhqOSzs',
  project='proj_2fII7izwVGgYaERNNKhhMx4l',
)

Debo cargar mi dataset de TEST y pasarle un archivo BATCH para que lo procese

Usare este mensaje para cada Tweet:

"role": "system", "content": "You are trained to analyze and detect the sentiment of the given text. Only hateful or not hateful is allowed. The answer should be in spanish"

In [80]:
from datasets import load_dataset
import pandas as pd

ds1 = load_dataset("piuba-bigdata/contextualized_hate_speech")


In [81]:
ds1

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 36420
    })
    test: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 11343
    })
    dev: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 9106
    })
})

In [82]:
#tomo una muestra de 100 TW random
df_test = pd.DataFrame(ds1['test'])
df_test

,id,title,text,context_tweet,HATEFUL,body,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL
0,397703,Mora Godoy cierra su escuela de tango y remata...,@usuario Ojalá se funda y cague de hambre,Mora Godoy cierra su escuela de tango y remata...,0,Puede marcar con justeza la fecha de inicio de...,0,0,0,0,0,0,0,0,0
1,397704,Mora Godoy cierra su escuela de tango y remata...,@usuario Faltó que se enganchó al hijo de sofo...,Mora Godoy cierra su escuela de tango y remata...,1,Puede marcar con justeza la fecha de inicio de...,0,1,0,0,0,0,0,0,0
2,397705,Mora Godoy cierra su escuela de tango y remata...,"@usuario y bue, que le vaya a llorar a Cristin...",Mora Godoy cierra su escuela de tango y remata...,0,Puede marcar con justeza la fecha de inicio de...,0,0,0,0,0,0,0,0,0
3,397706,Mora Godoy cierra su escuela de tango y remata...,@usuario @usuario Jodete. Macri te hizo conoce...,Mora Godoy cierra su escuela de tango y remata...,0,Puede marcar con justeza la fecha de inicio de...,0,0,0,0,0,0,0,0,0
4,397707,Mora Godoy cierra su escuela de tango y remata...,"@usuario Que se joda, es su gobierno el que el...",Mora Godoy cierra su escuela de tango y remata...,0,Puede marcar con justeza la fecha de inicio de...,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11338,361071,Nahir Galarza sumó a una abogada feminista a s...,@usuario Con esa varita q no rompe un plato ni...,Nahir Galarza sumó a una abogada feminista a s...,0,"La defensa de Nahir Galarza, condenada el 3 de...",0,0,0,0,0,0,0,0,0
11339,361072,Nahir Galarza sumó a una abogada feminista a s...,@usuario No las entiendo a estas pelotudas. So...,Nahir Galarza sumó a una abogada feminista a s...,1,"La defensa de Nahir Galarza, condenada el 3 de...",0,1,0,0,0,0,0,0,0
11340,361073,Nahir Galarza sumó a una abogada feminista a s...,@usuario Lo mató por qué es macho!,Nahir Galarza sumó a una abogada feminista a s...,0,"La defensa de Nahir Galarza, condenada el 3 de...",0,0,0,0,0,0,0,0,0
11341,361074,Nahir Galarza sumó a una abogada feminista a s...,@usuario Cuando sos asesina tenés que hacerte ...,Nahir Galarza sumó a una abogada feminista a s...,0,"La defensa de Nahir Galarza, condenada el 3 de...",0,0,0,0,0,0,0,0,0


Documentacion para el trabajo en batch:
https://platform.openai.com/docs/guides/batch/getting-started


Se prepara el formato completo del archivo para batch con el siguiente formato:

{"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}


Al rol System se le envía la siguiente consigna:

You must classify a text with one or more of the following labels: Label WOMEN discrimination against women, label LGBTI gender discrimination, specifically against gays, lesbians or transsexuals,label RACISM is racism, or discrimination against people according to their skin color,label
CLASS is discrimination for being poor or living in a marginal or poor place, label POLITICS is hatred of politicians, label DISABLED is discrimination against people with disabilities, label APPEARENCE is discrimination against people because of their physical appearance, specifically fatphobia, label CRIMINAL is discrimination against people because of a criminal record or links to criminality and label CALLS is hateful text that incites violence
If the text is not hatefull indicate label NOT HATEFUL

In [83]:
def create_batch_multiple(df):
  # Specify the columns
  id_column = 'id'
  text_column = 'text'

  # Convert the dataframe into a list of dictionaries with custom_id and messages
  batch = [
      {
          'custom_id': str(row[id_column]),
          'method': 'POST',
          'url': '/v1/chat/completions',
          'body':{
              'model': 'gpt-4o-mini',
              'messages': [
                  {'role': 'system', 'content': 'You are trained to analyze and detect the sentiment of the given text.You must classify a text with one or more of the following labels: WOMEN when there is discrimination against women, LGBTI when there is discrimination against gays, lesbians or transsexuals, RACISM when there is discrimination against people according to their skin color, CLASS  when there is discrimination for being poor or living in a marginal or poor place, POLITICS is hatred of politicians, DISABLED when there is discrimination against people with disabilities, APPEARENCE when there is discrimination against people because of their physical appearance, specifically fatphobia, CRIMINAL when there is discrimination against people because of a criminal record or links to criminality and  CALLS is hateful text that incites violence. If the text is not hateful indicate NOT HATEFUL'},
                  {'role': 'user', 'content': row[text_column]}
                  ],
              'max_tokens': 20,  # Limit response to a single word
              'temperature': 0  # Keep response consistent
          }
      }
      for _, row in df.iterrows()
  ]
  return(batch)

Como no puedo hacer un solo batch porque se excede el limite voy a probar con trabajos de mitad

In [84]:
size_df=len(df_test)
size_df=int(size/2)
size_df

2836

In [85]:
df1 = df_test.iloc[:size_df,:]
df2 = df_test.iloc[size_df:,:]

In [86]:
df2[1:10]

,id,title,text,context_tweet,HATEFUL,body,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL
2837,364652,"Jonatan Viale, tras el retuit de Alberto Ferná...",@usuario Y tampoco seudos periodismo tan tira ...,"Jonatan Viale, tras los agravios de Alberto Fe...",0,"""Si no entendemos ahora los riesgos de una aut...",0,0,0,0,0,0,0,0,0
2838,364653,"Jonatan Viale, tras el retuit de Alberto Ferná...",@usuario Todo esto sirvió en 2015...ahora no s...,"Jonatan Viale, tras los agravios de Alberto Fe...",0,"""Si no entendemos ahora los riesgos de una aut...",0,0,0,0,0,0,0,0,0
2839,364654,"Jonatan Viale, tras el retuit de Alberto Ferná...",@usuario Tampoco un gordito lechoso rabioso. S...,"Jonatan Viale, tras los agravios de Alberto Fe...",1,"""Si no entendemos ahora los riesgos de una aut...",0,0,0,0,0,0,0,1,0
2840,364655,"Jonatan Viale, tras el retuit de Alberto Ferná...",@usuario Q caradura este tipoooo,"Jonatan Viale, tras los agravios de Alberto Fe...",0,"""Si no entendemos ahora los riesgos de una aut...",0,0,0,0,0,0,0,0,0
2841,364656,"Jonatan Viale, tras el retuit de Alberto Ferná...",@usuario Viale no es rabioso Alberto sabe lo q...,"Jonatan Viale, tras los agravios de Alberto Fe...",0,"""Si no entendemos ahora los riesgos de una aut...",0,0,0,0,0,0,0,0,0
2842,364657,"Jonatan Viale, tras el retuit de Alberto Ferná...",@usuario a todo esto...mauro viale... por quié...,"Jonatan Viale, tras los agravios de Alberto Fe...",0,"""Si no entendemos ahora los riesgos de una aut...",0,0,0,0,0,0,0,0,0
2843,364658,"Jonatan Viale, tras el retuit de Alberto Ferná...",@usuario Mucho menos un terrorista de medios,"Jonatan Viale, tras los agravios de Alberto Fe...",0,"""Si no entendemos ahora los riesgos de una aut...",0,0,0,0,0,0,0,0,0
2844,364659,"Jonatan Viale, tras el retuit de Alberto Ferná...",@usuario La sociedad no quiere operadores y sa...,"Jonatan Viale, tras los agravios de Alberto Fe...",0,"""Si no entendemos ahora los riesgos de una aut...",0,0,0,0,0,0,0,0,0
2845,356091,"Beatriz Sarlo: ""Me equivoqué con Cristina Kirc...",@usuario #AlbertoFernandezDictador,"Beatriz Sarlo: ""Me equivoqué con Cristina Kirc...",0,La intelectual señaló un error en la interpret...,0,0,0,0,0,0,0,0,0


In [87]:
batch1=create_batch_multiple(df1)
batch1[0]

{'custom_id': '397703',
 'method': 'POST',
 'url': '/v1/chat/completions',
 'body': {'model': 'gpt-4o-mini',
  'messages': [{'role': 'system',
    'content': 'You are trained to analyze and detect the sentiment of the given text.You must classify a text with one or more of the following labels: WOMEN when there is discrimination against women, LGBTI when there is discrimination against gays, lesbians or transsexuals, RACISM when there is discrimination against people according to their skin color, CLASS  when there is discrimination for being poor or living in a marginal or poor place, POLITICS is hatred of politicians, DISABLED when there is discrimination against people with disabilities, APPEARENCE when there is discrimination against people because of their physical appearance, specifically fatphobia, CRIMINAL when there is discrimination against people because of a criminal record or links to criminality and  CALLS is hateful text that incites violence. If the text is not hateful 

In [88]:
batch2=create_batch_multiple(df2)
batch2[0]

{'custom_id': '364651',
 'method': 'POST',
 'url': '/v1/chat/completions',
 'body': {'model': 'gpt-4o-mini',
  'messages': [{'role': 'system',
    'content': 'You are trained to analyze and detect the sentiment of the given text.You must classify a text with one or more of the following labels: WOMEN when there is discrimination against women, LGBTI when there is discrimination against gays, lesbians or transsexuals, RACISM when there is discrimination against people according to their skin color, CLASS  when there is discrimination for being poor or living in a marginal or poor place, POLITICS is hatred of politicians, DISABLED when there is discrimination against people with disabilities, APPEARENCE when there is discrimination against people because of their physical appearance, specifically fatphobia, CRIMINAL when there is discrimination against people because of a criminal record or links to criminality and  CALLS is hateful text that incites violence. If the text is not hateful 

In [89]:
# Save the batch to a .jsonl file
with open('batch_gpt_multiclass_1.jsonl', 'w') as f:
    for entry in batch1:
        json.dump(entry, f)
        f.write('\n')

print("Batch file created successfully in .jsonl format!")

Batch file created successfully in .jsonl format!


In [90]:
batch_input_file = client.files.create(
  file=open("batch_gpt_multiclass_1.jsonl", "rb"),
  purpose="batch"
)

In [91]:
batch_input_file

FileObject(id='file-8De93SpUWlHdHxnfKJkjsUHK', bytes=3342020, created_at=1721946356, filename='batch_gpt_multiclass_1.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [92]:
batch_input_file_id = 'file-8De93SpUWlHdHxnfKJkjsUHK'

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "nightly eval job"
    }
)

Batch(id='batch_ywUPqt2pJygVz0rsnjnjDQdH', completion_window='24h', created_at=1721946376, endpoint='/v1/chat/completions', input_file_id='file-8De93SpUWlHdHxnfKJkjsUHK', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1722032776, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [93]:
client.batches.retrieve('batch_ywUPqt2pJygVz0rsnjnjDQdH')

Batch(id='batch_ywUPqt2pJygVz0rsnjnjDQdH', completion_window='24h', created_at=1721946376, endpoint='/v1/chat/completions', input_file_id='file-8De93SpUWlHdHxnfKJkjsUHK', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1722032776, failed_at=None, finalizing_at=None, in_progress_at=1721946377, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=134, failed=0, total=2836))

In [ ]:
batch_NQ9RgzBbLyqfmLOnoBYq0OPG

Ahora creo el segundo batch

In [98]:
with open('batch_gpt_multiclass_2.jsonl', 'w') as f:
    for entry in batch2:
        json.dump(entry, f)
        f.write('\n')

print("Batch file created successfully in .jsonl format!")

Batch file created successfully in .jsonl format!


In [99]:
batch_input_file = client.files.create(
  file=open("batch_gpt_multiclass_2.jsonl", "rb"),
  purpose="batch"
)
batch_input_file

FileObject(id='file-gQSXsyeUzGjx8SessxovIhN7', bytes=10037286, created_at=1721947752, filename='batch_gpt_multiclass_2.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [102]:
batch_input_file_id = 'file-gQSXsyeUzGjx8SessxovIhN7'

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "nightly eval job"
    }
)

Batch(id='batch_wmwRh7Ffw2zIrCi2D8Msz6PG', completion_window='24h', created_at=1721947784, endpoint='/v1/chat/completions', input_file_id='file-gQSXsyeUzGjx8SessxovIhN7', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1722034184, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [103]:
client.batches.retrieve('batch_wmwRh7Ffw2zIrCi2D8Msz6PG')

Batch(id='batch_wmwRh7Ffw2zIrCi2D8Msz6PG', completion_window='24h', created_at=1721947784, endpoint='/v1/chat/completions', input_file_id='file-gQSXsyeUzGjx8SessxovIhN7', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='token_limit_exceeded', line=None, message='Enqueued token limit reached for gpt-4o-mini in organization org-1uHjwiaB3OlPzoxfVzhqOSzs. Limit: 2,000,000 enqueued tokens. Please try again once some in_progress batches have been completed.', param=None)], object='list'), expired_at=None, expires_at=1722034184, failed_at=1721947790, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))